In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import RobustScaler
from ta import add_all_ta_features
from torch.nn.utils import weight_norm
import matplotlib.pyplot as plt
from pykalman import KalmanFilter

# Check for CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ================== 1. Enhanced Data Pipeline ==================
def create_sequences(data, targets, window_size, horizon):
    X, Y = [], []
    for i in range(len(data) - window_size - horizon + 1):  # Fixed off-by-one error
        X.append(data[i:i+window_size])
        Y.append(targets[i+window_size:i+window_size+horizon])
    return np.array(X), np.array(Y)

# Load data with proper error handling
try:
    df = pd.read_csv("AAPL_stock_data_2020_01_01.csv")
    # Verify the data has expected columns
    required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"CSV is missing required columns. Found: {df.columns.tolist()}")
except Exception as e:
    print(f"Error loading data: {e}")
    raise

# Clean and prepare the data
# Assuming first column contains dates
date_col = df.columns[0]
df = df.rename(columns={date_col: 'Date'})
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # Handle date parsing errors gracefully
df = df.dropna(subset=['Date'])  # Remove rows with invalid dates
df = df.set_index('Date')

# Ensure numeric data types for critical columns
for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Fill any remaining NaN values using forward fill then backward fill
df = df.fillna(method='ffill').fillna(method='bfill')

# Add comprehensive technical features with error handling
try:
    df = add_all_ta_features(
        df, open="Open", high="High", low="Low", close="Close", volume="Volume"
    )
except Exception as e:
    print(f"Error adding technical indicators: {e}")
    # Fallback to basic features if technical indicators fail
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['RSI'] = compute_rsi(df['Close'], 14)  # We'll define this function below

# Define RSI calculation as fallback
def compute_rsi(prices, window=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0).rolling(window=window).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=window).mean()
    
    # Handle zero division case
    rs = gain / loss.replace(0, np.finfo(float).eps)
    return 100 - (100 / (1 + rs))

# Feature selection (choosing robust features)
features = [
    'Close', 'High', 'Low', 'Open', 'Volume'
]

# Add technical indicators if they exist
ta_features = [
    'momentum_rsi', 'volatility_bbh', 'trend_macd',
    'volume_obv', 'volatility_atr', 'trend_ema_fast', 'trend_ema_slow'
]

# Only include features that exist in the dataframe
features = [f for f in features if f in df.columns]
ta_features = [f for f in ta_features if f in df.columns]
features.extend(ta_features)

# Check if we have enough features
if len(features) < 5:
    raise ValueError(f"Not enough features available. Only found: {features}")

# Verify data quality
if df[features].isna().any().any():
    # Final safety check for NaNs, fill with column means
    df[features] = df[features].fillna(df[features].mean())

target = 'Close'

# ================== 2. Advanced Normalization ==================
# Ensure we have enough data
if len(df) < 30:
    raise ValueError(f"Insufficient data points: {len(df)} (minimum 30 required)")

train_size = int(0.7 * len(df))
val_size = int(0.15 * len(df))

# Use robust scaler to handle outliers
scaler = RobustScaler()
target_scaler = RobustScaler()  # Separate scaler for target

train_data = scaler.fit_transform(df.iloc[:train_size][features])
val_data = scaler.transform(df.iloc[train_size:train_size+val_size][features])
test_data = scaler.transform(df.iloc[train_size+val_size:][features])

# Scale targets - fixed the target variable handling
train_targets = target_scaler.fit_transform(df.iloc[:train_size][[target]])
val_targets = target_scaler.transform(df.iloc[train_size:train_size+val_size][[target]])
test_targets = target_scaler.transform(df.iloc[train_size+val_size:][[target]])

# ================== 3. Temporal Convolutional Network ==================
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = nn.Sequential(
            weight_norm(nn.Conv1d(input_size, num_channels, kernel_size, padding=(kernel_size-1)//2 + (kernel_size-1)%2)),  # Fixed padding
            nn.ReLU(),
            nn.Dropout(dropout),
            weight_norm(nn.Conv1d(num_channels, num_channels*2, kernel_size, padding=(kernel_size-1)//2 + (kernel_size-1)%2)),
            nn.ReLU(),
            nn.Dropout(dropout),
            weight_norm(nn.Conv1d(num_channels*2, output_size, kernel_size, padding=(kernel_size-1)//2 + (kernel_size-1)%2))
        )
        self.init_weights()
        
        # Add a linear layer to ensure output size matches horizon
        self.linear = nn.Linear(output_size, output_size)

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    def forward(self, x):
        # Input shape: [batch, sequence, features]
        x = x.permute(0, 2, 1)  # [batch, features, sequence]
        
        # Apply TCN layers
        y = self.tcn(x)
        
        # Ensure we get the right-most output of the correct length
        y = y[:, :, -1]
        
        # Apply final linear layer
        y = self.linear(y)
        
        return y

# ================== 4. Hybrid Loss Function ==================
class TemporalLoss(nn.Module):
    def __init__(self, alpha=0.7):
        super().__init__()
        self.alpha = alpha
        self.mse = nn.MSELoss()
        self.mae = nn.L1Loss()
        
    def forward(self, preds, targets):
        mse_loss = self.mse(preds, targets)
        
        # Prevent issues with batches of size 1 or prediction horizons of 1
        if preds.size(1) > 1:
            temporal_loss = self.mae(preds[:, 1:] - preds[:, :-1], 
                                   targets[:, 1:] - targets[:, :-1])
        else:
            temporal_loss = torch.tensor(0.0, device=preds.device)
            
        return self.alpha * mse_loss + (1 - self.alpha) * temporal_loss

# ================== 5. Training Protocol ==================
window_size = 21
horizon = 3  # Predict next 3 days

# Create sequences with error handling
try:
    X_train, y_train = create_sequences(train_data, train_targets, window_size, horizon)
    X_val, y_val = create_sequences(val_data, val_targets, window_size, horizon)
    X_test, y_test = create_sequences(test_data, test_targets, window_size, horizon)
    
    # Safety check for sequence creation
    if len(X_train) == 0 or len(X_val) == 0 or len(X_test) == 0:
        raise ValueError("Failed to create sequences - insufficient data")
    
    # Fix target shapes - ensure they are 2D (batch_size, horizon)
    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1])
    y_val = y_val.reshape(y_val.shape[0], y_val.shape[1])
    y_test = y_test.reshape(y_test.shape[0], y_test.shape[1])
        
except Exception as e:
    print(f"Error in sequence creation: {e}")
    raise

# Initialize model with error handling
try:
    model = TCN(
        input_size=len(features), 
        output_size=horizon, 
        num_channels=64, 
        kernel_size=5, 
        dropout=0.4
    ).to(device)
    
    # Use a more robust optimizer
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
    loss_fn = TemporalLoss(alpha=0.6)
    
except Exception as e:
    print(f"Error initializing model: {e}")
    raise

# ================== 6. Advanced Training Loop with Error Handling ==================
try:
    # Convert data to PyTorch tensors once to avoid repeated conversion
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

    # Training parameters
    batch_size = 32
    epochs = 200
    patience = 20
    best_val_loss = float('inf')
    counter = 0
    
    # Training loop with early stopping
    for epoch in range(epochs):
        model.train()
        train_losses = []
        
        # Create random indices for shuffling
        indices = torch.randperm(len(X_train_tensor))
        
        # Process in batches
        for i in range(0, len(indices), batch_size):
            # Get batch indices
            batch_indices = indices[i:i+batch_size]
            
            # Get batch data
            x_batch = X_train_tensor[batch_indices]
            y_batch = y_train_tensor[batch_indices]
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(x_batch)
            
            # Ensure outputs and targets have same shape
            if outputs.shape != y_batch.shape:
                raise ValueError(f"Shape mismatch: outputs {outputs.shape}, targets {y_batch.shape}")
                
            # Calculate loss
            loss = loss_fn(outputs, y_batch)
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # Update weights
            optimizer.step()
            
            # Record loss
            train_losses.append(loss.item())
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = loss_fn(val_outputs, y_val_tensor)
            
        # Update learning rate based on validation loss
        scheduler.step(val_loss)
        
        # Print progress
        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Train Loss {np.mean(train_losses):.4f}, Val Loss {val_loss.item():.4f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            # Save best model
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
    
    # Load best model
    model.load_state_dict(torch.load('best_model.pth'))
    
except Exception as e:
    print(f"Error during training: {e}")
    # Save checkpoint if possible
    try:
        torch.save(model.state_dict(), 'emergency_model.pth')
        print("Saved emergency checkpoint")
    except:
        print("Failed to save emergency checkpoint")
    raise

# ================== 7. Final Ensemble Prediction with Uncertainty Quantification ==================
try:
    model.eval()
    
    # Create uncertainty-aware predictions using Monte Carlo Dropout
    test_preds = []
    model.train()  # Set to train mode to enable dropout for MC sampling
    
    n_samples = 10
    for _ in range(n_samples):
        with torch.no_grad():
            pred = model(X_test_tensor).cpu().numpy()
            test_preds.append(pred)
    
    # Calculate statistics
    final_preds = np.median(test_preds, axis=0)
    pred_std = np.std(test_preds, axis=0)
    
    # Calculate confidence intervals (95%)
    lower_bound = np.percentile(test_preds, 2.5, axis=0)
    upper_bound = np.percentile(test_preds, 97.5, axis=0)
    
    # Inverse transform predictions back to original scale
    final_preds_orig = target_scaler.inverse_transform(final_preds.reshape(-1, 1)).reshape(final_preds.shape)
    lower_bound_orig = target_scaler.inverse_transform(lower_bound.reshape(-1, 1)).reshape(lower_bound.shape)
    upper_bound_orig = target_scaler.inverse_transform(upper_bound.reshape(-1, 1)).reshape(upper_bound.shape)
    y_test_orig = target_scaler.inverse_transform(y_test.reshape(-1, 1)).reshape(y_test.shape)
    
except Exception as e:
    print(f"Error during prediction: {e}")
    raise

# ================== 8. Kalman Filter Post-processing ==================
try:
    # Initialize Kalman Filter for smoothing predictions
    kf = KalmanFilter(
        initial_state_mean=final_preds_orig[0],
        initial_state_covariance=np.eye(horizon) * 0.01,
        transition_matrices=np.eye(horizon),
        observation_matrices=np.eye(horizon),
        observation_covariance=0.1 * np.eye(horizon),
        transition_covariance=0.01 * np.eye(horizon)
    )
    
    # Apply Kalman Filter smoothing
    smoothed_state_means, _ = kf.smooth(final_preds_orig)
    
    # Final predictions after filtering
    final_filtered_preds = smoothed_state_means
    
except Exception as e:
    print(f"Error during Kalman filtering, using unfiltered predictions: {e}")
    final_filtered_preds = final_preds_orig

# ================== 9. Visualization and Evaluation ==================
# Function to calculate metrics
def calculate_metrics(actual, predicted):
    mse = np.mean((actual - predicted) ** 2)
    mae = np.mean(np.abs(actual - predicted))
    mape = np.mean(np.abs((actual - predicted) / np.maximum(0.0001, np.abs(actual)))) * 100
    return {
        'MSE': mse,
        'RMSE': np.sqrt(mse),
        'MAE': mae,
        'MAPE': mape
    }

# Calculate metrics for each day in horizon
metrics_by_day = []
overall_metrics = {}
for i in range(horizon):
    day_metrics = calculate_metrics(
        y_test_orig[:, i].flatten(), 
        final_filtered_preds[:, i].flatten()
    )
    metrics_by_day.append(day_metrics)
    print(f"Day {i+1} Metrics: {day_metrics}")
    
    # Store Day 1 metrics for main visualization
    if i == 0:
        overall_metrics = day_metrics

test_rmse = overall_metrics['RMSE']
test_mape = overall_metrics['MAPE']

# Create visualization directory if it doesn't exist
import os
if not os.path.exists('visualizations'):
    os.makedirs('visualizations')

# ===== VISUALIZATION 1: SIMPLE COMPARISON PLOT =====
plt.figure(figsize=(14, 7))
plt.plot(y_test_orig[:, 0], label='Actual Prices', linewidth=2)
plt.plot(final_filtered_preds[:, 0], label=f'Predicted Prices (RMSE: {test_rmse:.2f})', alpha=0.7, linewidth=2)
plt.title('AAPL Stock Price Prediction Performance', fontsize=16)
plt.xlabel('Test Sample Index', fontsize=12)
plt.ylabel('Stock Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.savefig('visualizations/simple_prediction_comparison.png')
plt.close()

# ===== VISUALIZATION 2: MULTI-DAY FORECAST =====
plt.figure(figsize=(16, 8))
for i in range(horizon):
    day_rmse = metrics_by_day[i]['RMSE']
    plt.plot(y_test_orig[:, i], '-', label=f'Actual (Day {i+1})', linewidth=2)
    plt.plot(final_filtered_preds[:, i], '--', label=f'Predicted (Day {i+1}, RMSE: {day_rmse:.2f})', linewidth=2)

plt.title('Multi-Day Stock Price Forecast', fontsize=16)
plt.xlabel('Test Sample Index', fontsize=12)
plt.ylabel('Stock Price ($)', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.savefig('visualizations/multi_day_forecast.png')
plt.close()

# ===== VISUALIZATION 3: PREDICTION WITH CONFIDENCE INTERVALS =====
plt.figure(figsize=(15, 8))
plt.plot(y_test_orig[:, 0], 'k-', label='Actual Prices', linewidth=2)
plt.plot(final_filtered_preds[:, 0], 'b-', label=f'Predicted Prices (RMSE: {test_rmse:.2f})', linewidth=2)
plt.fill_between(
    range(len(y_test_orig)),
    lower_bound_orig[:, 0],
    upper_bound_orig[:, 0],
    color='blue',
    alpha=0.2,
    label='95% Confidence Interval'
)
plt.title('Stock Price Prediction with Uncertainty Bounds', fontsize=16)
plt.xlabel('Test Sample Index', fontsize=12)
plt.ylabel('Stock Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.savefig('visualizations/prediction_with_confidence.png')
plt.close()

# ===== VISUALIZATION 4: ZOOM IN ON SPECIFIC SECTION =====
# Focus on last 30 days or fewer if not enough data
zoom_range = min(30, len(y_test_orig))
plt.figure(figsize=(15, 8))
plt.plot(y_test_orig[-zoom_range:, 0], 'r-', label='Actual Prices', linewidth=2.5)
plt.plot(final_filtered_preds[-zoom_range:, 0], 'g-', label=f'Predicted Prices', linewidth=2.5)
plt.fill_between(
    range(zoom_range),
    lower_bound_orig[-zoom_range:, 0],
    upper_bound_orig[-zoom_range:, 0],
    color='green',
    alpha=0.2,
    label='95% Confidence Interval'
)

# Add error metrics to plot
plt.text(
    0.02, 0.95, 
    f'RMSE: {test_rmse:.2f}\nMAPE: {test_mape:.2f}%', 
    transform=plt.gca().transAxes,
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.8)
)

plt.title('Recent Stock Price Predictions (Last 30 Days)', fontsize=16)
plt.xlabel('Days', fontsize=12)
plt.ylabel('Stock Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.savefig('visualizations/recent_predictions.png')
plt.close()

# ===== VISUALIZATION 5: ERROR ANALYSIS =====
errors = y_test_orig[:, 0] - final_filtered_preds[:, 0]
plt.figure(figsize=(15, 10))

# Error distribution
plt.subplot(2, 1, 1)
plt.hist(errors, bins=30, alpha=0.7, color='b')
plt.axvline(x=0, color='r', linestyle='--', linewidth=2)
plt.title('Prediction Error Distribution', fontsize=16)
plt.xlabel('Error Value', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True, alpha=0.3)

# Error over time
plt.subplot(2, 1, 2)
plt.plot(errors, 'b-', label='Prediction Error', linewidth=2)
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.title('Prediction Error Over Time', fontsize=16)
plt.xlabel('Test Sample Index', fontsize=12)
plt.ylabel('Error Value', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('visualizations/error_analysis.png')
plt.close()

print("Visualizations saved to 'visualizations' directory.")

# Plot results - as requested in the exact format
plt.figure(figsize=(14, 7))
plt.plot(y_test_orig[:, 0], label='Actual Prices', linewidth=2)
plt.plot(final_filtered_preds[:, 0], label=f'Predicted Prices (RMSE: {test_rmse:.2f})', alpha=0.7, linewidth=2)
plt.title(f'AAPL Price Prediction (Test RMSE: {test_rmse:.2f})', fontsize=16)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.savefig('visualizations/requested_visualization.png')
plt.show()  # This will display the plot if running in an interactive environment


print("Analysis complete. Results saved.")

# ================== 10. Enhanced Investment Returns Analysis with Risk Management ==================
# Replace the original calculate_investment_returns function with this enhanced version
def calculate_investment_returns(actual_prices, predicted_prices, prediction_std=None, initial_investment=100.0, trading_fee_percent=0.1):
    """
    Calculate investment returns using a strategy with multiple risk management techniques:
    - Position sizing based on prediction confidence
    - Stop-loss implementation
    - Volatility-based adjustment
    - Trend following filter
    - Lower exposure during uncertain markets
    """
    # Initialize variables
    cash = initial_investment
    shares = 0
    portfolio_values = [initial_investment]  # Starting portfolio value
    actions = []  # Track buy/sell/hold actions
    
    # Use actual close prices for valuation
    actual_close_prices = actual_prices[:, 0]
    day1_predictions = predicted_prices[:, 0]
    try:
        # Convert list of predictions to 3D numpy array
        # Shape: (num_ensemble_runs, num_samples, prediction_horizon)
        test_preds_array = np.array(test_preds)
        
        # Verify we have a 3D array
        if test_preds_array.ndim != 3:
            raise ValueError("Predictions should be 3D array: (runs, samples, horizon)")
        
        # Calculate std for first prediction step across all runs
        prediction_std = np.std(test_preds_array[:, :, 0], axis=0)  # Shape: (num_samples,)
        print(f"Using prediction std with shape: {prediction_std.shape}")
    finally:    
        # If no prediction standard deviation is provided, create a default one
        if prediction_std is None:
            prediction_std = np.ones_like(day1_predictions) * 0.01
       
    # Add validation checks at the start of the function
    
    if prediction_std is not None:
        assert len(prediction_std) == len(actual_prices), \
            "prediction_std must match actual_prices length"
        assert prediction_std.ndim == 1, \
            "prediction_std must be 1D array"
        
    # Track entries for stop loss calculation
    entry_prices = []
    max_drawdown = 0
    peak_value = initial_investment
    
    # Risk management parameters
    max_position_pct = 0.6  # Maximum position size as percentage of portfolio
    stop_loss_pct = 0.05  # 5% stop loss
    trailing_stop_pct = 0.07  # 7% trailing stop loss
    profit_take_pct = 0.1  # 10% profit taking
    confidence_threshold = 0.01  # Minimum predicted change to trigger a trade
    
    # For market regime detection
    window_size = min(20, len(actual_close_prices) - 1)  # Use smaller window if not enough data
    
    for i in range(len(actual_close_prices) - 1):
        current_price = actual_close_prices[i]
        
        # Calculate current portfolio value
        current_portfolio_value = cash + shares * current_price
        
        # Update maximum drawdown
        if current_portfolio_value > peak_value:
            peak_value = current_portfolio_value
        current_drawdown = (peak_value - current_portfolio_value) / peak_value * 100
        max_drawdown = max(max_drawdown, current_drawdown)
        
        # Skip first day (we need previous data)
        if i <= 0:
            actions.append("Day 0: Initial position")
            portfolio_values.append(current_portfolio_value)
            continue
        
        # ===== RISK MANAGEMENT TECHNIQUE 1: Market Regime Detection =====
        # Simple moving average crossover for trend detection
        if i >= window_size:
            short_ma = np.mean(actual_close_prices[i-window_size//2:i])
            long_ma = np.mean(actual_close_prices[i-window_size:i])
            market_trend = "UPTREND" if short_ma > long_ma else "DOWNTREND"
        else:
            # Not enough data, assume neutral
            market_trend = "NEUTRAL" 
        
        # ===== RISK MANAGEMENT TECHNIQUE 2: Volatility Assessment =====
        # Calculate recent volatility
        if i >= window_size:
            recent_returns = np.diff(actual_close_prices[i-window_size:i]) / actual_close_prices[i-window_size:i-1]
            current_volatility = np.std(recent_returns)
            avg_volatility = 0.015  # Typical stock daily volatility
            volatility_ratio = current_volatility / avg_volatility
        else:
            volatility_ratio = 1.0  # Default to normal
        
        # ===== RISK MANAGEMENT TECHNIQUE 3: Confidence-Based Trading =====
        # What our model predicted for tomorrow
        predicted_next_price = day1_predictions[i]
        predicted_change_percent = (predicted_next_price - current_price) / current_price
        
        # Modify confidence calculation to prevent division by zero
        prediction_confidence = 1.0 / (1.0 + prediction_std[i] + 1e-8)  # Add epsilon        
        
        # Only trade if the predicted change exceeds our threshold and aligns with market trend
        valid_signal = abs(predicted_change_percent) > confidence_threshold
        
        # ===== RISK MANAGEMENT TECHNIQUE 4: Stop Loss Check =====
        # Check if we need to exit due to stop loss
        stop_loss_triggered = False
        if shares > 0 and len(entry_prices) > 0:
            # Calculate average entry price
            avg_entry_price = sum(entry_prices) / len(entry_prices)
            
            # Fixed stop loss
            if current_price < avg_entry_price * (1 - stop_loss_pct):
                stop_loss_triggered = True
                actions.append(f"Day {i}: STOP LOSS TRIGGERED at ${current_price:.2f} (Entry: ${avg_entry_price:.2f})")
            
            # Trailing stop loss
            elif shares > 0:
                # Calculate highest price since entry
                highest_since_entry = max(actual_close_prices[i-len(entry_prices):i+1])
                if current_price < highest_since_entry * (1 - trailing_stop_pct):
                    stop_loss_triggered = True
                    actions.append(f"Day {i}: TRAILING STOP TRIGGERED at ${current_price:.2f} (High: ${highest_since_entry:.2f})")
            
            # Take profit
            elif current_price > avg_entry_price * (1 + profit_take_pct):
                actions.append(f"Day {i}: PROFIT TARGET REACHED at ${current_price:.2f}")
                stop_loss_triggered = True  # Use same mechanism to exit
        
        # Execute stop loss if triggered
        if stop_loss_triggered and shares > 0:
            # Sell all shares
            trade_value = shares * current_price
            fee = trade_value * (trading_fee_percent/100)
            
            # Update portfolio
            cash += (trade_value - fee)
            shares = 0
            entry_prices = []  # Reset entry prices
        
        # Regular trading logic with risk management
        elif valid_signal:
            # FIX: Ensure predicted_trend is a scalar boolean, not an array
            predicted_trend = float(predicted_next_price) > current_price
            
            # ===== RISK MANAGEMENT TECHNIQUE 5: Position Sizing =====
            # Adjust position size based on:
            # 1. Prediction confidence
            # 2. Market volatility
            # 3. Market trend
            
            # Base position size as percentage of portfolio
            base_position_pct = max_position_pct * prediction_confidence
            
            # Reduce position in high volatility
            if volatility_ratio > 1.5:
                position_pct = base_position_pct * 0.5
            elif volatility_ratio > 1.2:
                position_pct = base_position_pct * 0.75
            else:
                position_pct = base_position_pct
            
            # Consider market trend - reduce position if trading against trend
            if (predicted_trend and market_trend == "DOWNTREND") or \
               (not predicted_trend and market_trend == "UPTREND"):
                position_pct *= 0.5  # Reduce position when trading against trend
            
            # Execute trade with adjusted position size
            if predicted_trend and cash > 0:  # Model predicts price will go up, buy
                # Calculate position size
                # Modify the cash_to_use calculation in the trading loop
                cash_to_use = float(current_portfolio_value * position_pct)
                cash_to_use = min(cash_to_use, cash)  # Now comparing two scalars
                if cash_to_use > 0:
                    # Calculate shares to buy with adjusted cash amount
                    shares_to_buy = cash_to_use / (current_price * (1 + trading_fee_percent/100))
                    
                    trade_cost = shares_to_buy * current_price
                    fee = trade_cost * (trading_fee_percent/100)
                    
                    # Update portfolio
                    shares += shares_to_buy
                    cash -= (trade_cost + fee)
                    actions.append(f"Day {i}: BUY {shares_to_buy:.4f} shares at ${current_price:.2f} (Confidence: {prediction_confidence:.2f}, Volatility: {volatility_ratio:.1f}X)")
                    
                    # Record entry for stop loss
                    entry_prices.append(current_price)
                else:
                    actions.append(f"Day {i}: HOLD - Signal too weak to trade")
                    
            elif not predicted_trend and shares > 0:  # Model predicts price will go down, sell
                # Calculate position size to sell (partial position)
                shares_to_sell = shares * position_pct
                
                if shares_to_sell > 0:
                    trade_value = shares_to_sell * current_price
                    fee = trade_value * (trading_fee_percent/100)
                    
                    # Update portfolio
                    cash += (trade_value - fee)
                    shares -= shares_to_sell
                    actions.append(f"Day {i}: SELL {shares_to_sell:.4f} shares at ${current_price:.2f} (Confidence: {prediction_confidence:.2f})")
                    
                    # Update entry prices if partial sale
                    if shares > 0 and len(entry_prices) > 0:
                        # Remove corresponding proportion of entry prices
                        num_entries_to_remove = int(len(entry_prices) * (shares_to_sell / (shares + shares_to_sell)))
                        entry_prices = entry_prices[num_entries_to_remove:]
                else:
                    actions.append(f"Day {i}: HOLD - Signal too weak to trade")
                    
            else:
                actions.append(f"Day {i}: HOLD (Cash: ${cash:.2f}, Shares: {shares:.4f})")
        else:
            actions.append(f"Day {i}: HOLD - No clear signal")
            
        # Record portfolio value for this day
        portfolio_values.append(cash + shares * current_price)
    
    # Calculate final portfolio value and return metrics
    initial_value = portfolio_values[0]
    final_value = portfolio_values[-1]
    total_return = ((final_value / initial_value) - 1) * 100
    
    # Calculate annualized return (assuming 252 trading days per year)
    days = len(portfolio_values) - 1
    annualized_return = (((final_value / initial_value) ** (252 / days)) - 1) * 100
    
    return {
        'initial_investment': initial_investment,
        'final_value': final_value,
        'total_return_percent': total_return,
        'annualized_return_percent': annualized_return,
        'max_drawdown_percent': max_drawdown,
        'portfolio_values': portfolio_values,
        'actions': actions
    }

# ===== Code to replace the existing investment analysis section =====
# Calculate investment returns with the new risk management strategy
print("\n=== Running Enhanced Risk-Managed Investment Analysis ===")

# We'll use the prediction standard deviation for confidence-based position sizing
# Extract standard deviation from the ensemble predictions if available
try:
    prediction_std = np.std(test_preds, axis=0)
    print(f"Using prediction standard deviation for confidence-based sizing")
except:
    prediction_std = None
    print(f"No prediction standard deviation available, using default confidence")

# First, calculate the original investment results (assuming this is already done previously)
# If not, uncomment the following:
investment_results = calculate_original_investment_returns(y_test_orig, final_filtered_preds, initial_investment=100.0)

# ===== Safe Prediction Standard Deviation Calculation =====
try:
    # Ensure we have numpy arrays
    if isinstance(test_preds, list):
        test_preds_array = np.stack(test_preds)  # Convert list of arrays to 3D array
    else:
        test_preds_array = test_preds.copy()
    
    # Verify array dimensions
    if test_preds_array.ndim == 2:
        # Handle single-run predictions (add ensemble dimension)
        test_preds_array = test_preds_array[np.newaxis, ...]
    
    # Calculate standard deviation for first prediction step
    prediction_std = np.std(test_preds_array[:, :, 0], axis=0)
    print(f"Successfully calculated prediction std (shape: {prediction_std.shape})")
    
except Exception as e:
    print(f"Prediction std calculation failed: {e}")
    prediction_std = np.ones(len(y_test_orig)) * 0.05  # Default 5% uncertainty
    print("Using default prediction standard deviation")

# Now use in your investment calculation
risk_managed_results = calculate_investment_returns(
    y_test_orig, 
    final_filtered_preds,
    prediction_std=prediction_std,
    initial_investment=100.0,
    trading_fee_percent=0.1
)
# Print investment metrics
print("\n=== Risk-Managed Investment Performance (Starting with $100) ===")
print(f"Final Portfolio Value: ${risk_managed_results['final_value']:.2f}")
print(f"Total Return: {risk_managed_results['total_return_percent']:.2f}%")
print(f"Annualized Return: {risk_managed_results['annualized_return_percent']:.2f}%")
print(f"Maximum Drawdown: {risk_managed_results['max_drawdown_percent']:.2f}%")
print(f"Drawdown Reduction: {investment_results['max_drawdown_percent'] - risk_managed_results['max_drawdown_percent']:.2f}%")

# Compare with original strategy
performance_ratio = risk_managed_results['total_return_percent'] / max(1.0, investment_results['total_return_percent'])
print(f"Performance ratio vs original: {performance_ratio:.2f}x")

# ===== Additional Visualization for Risk-Managed Strategy =====
plt.figure(figsize=(15, 10))

# Plot portfolio values for both strategies
plt.subplot(2, 1, 1)
plt.plot(investment_results['portfolio_values'], 'r-', label='Original Strategy', linewidth=2)
plt.plot(risk_managed_results['portfolio_values'], 'g-', label='Risk-Managed Strategy', linewidth=2)
plt.title('Portfolio Value Comparison', fontsize=16)
plt.xlabel('Trading Days', fontsize=12)
plt.ylabel('Portfolio Value ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

# Plot drawdowns for both strategies
plt.subplot(2, 1, 2)
original_drawdowns = []
managed_drawdowns = []

# Calculate drawdown series for original strategy
peak = investment_results['portfolio_values'][0]
for value in investment_results['portfolio_values']:
    if value > peak:
        peak = value
    drawdown = (peak - value) / peak * 100
    original_drawdowns.append(drawdown)

# Calculate drawdown series for risk-managed strategy
peak = risk_managed_results['portfolio_values'][0]
for value in risk_managed_results['portfolio_values']:
    if value > peak:
        peak = value
    drawdown = (peak - value) / peak * 100
    managed_drawdowns.append(drawdown)

plt.plot(original_drawdowns, 'r-', label=f'Original Drawdown (Max: {investment_results["max_drawdown_percent"]:.2f}%)', linewidth=2)
plt.plot(managed_drawdowns, 'g-', label=f'Managed Drawdown (Max: {risk_managed_results["max_drawdown_percent"]:.2f}%)', linewidth=2)
plt.title('Drawdown Comparison', fontsize=16)
plt.xlabel('Trading Days', fontsize=12)
plt.ylabel('Drawdown (%)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.gca().invert_yaxis()  # Invert Y axis to show drawdowns as negative values

plt.tight_layout()
plt.savefig('visualizations/risk_management_comparison.png')
plt.close()

# Create a summary plot for presentation
plt.figure(figsize=(15, 8))
plt.plot(risk_managed_results['portfolio_values'], 'g-', linewidth=2.5)
plt.title(f'Risk-Managed Portfolio Growth (Starting with $100)', fontsize=16)
plt.xlabel('Trading Days', fontsize=12)
plt.ylabel('Portfolio Value ($)', fontsize=12)
plt.grid(True, alpha=0.3)

# Add key metrics as text
plt.text(
    0.02, 0.85, 
    f'Final Value: ${risk_managed_results["final_value"]:.2f}\n'
    f'Total Return: {risk_managed_results["total_return_percent"]:.2f}%\n'
    f'Annualized: {risk_managed_results["annualized_return_percent"]:.2f}%\n'
    f'Max Drawdown: {risk_managed_results["max_drawdown_percent"]:.2f}%\n'
    f'Original Drawdown: {investment_results["max_drawdown_percent"]:.2f}%', 
    transform=plt.gca().transAxes,
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.8)
)

plt.tight_layout()
plt.savefig('visualizations/risk_managed_portfolio.png')
plt.show()  # This will display the plot if running in an interactive environment

# Save detailed trading log with risk-managed approach
with open('visualizations/risk_managed_trading_log.txt', 'w') as f:
    f.write("=== Risk-Managed Investment Trading Log ===\n")
    f.write(f"Initial Investment: ${risk_managed_results['initial_investment']:.2f}\n")
    f.write(f"Final Portfolio Value: ${risk_managed_results['final_value']:.2f}\n")
    f.write(f"Total Return: {risk_managed_results['total_return_percent']:.2f}%\n")
    f.write(f"Annualized Return: {risk_managed_results['annualized_return_percent']:.2f}%\n")
    f.write(f"Maximum Drawdown: {risk_managed_results['max_drawdown_percent']:.2f}%\n\n")
    
    f.write("=== Risk Management Improvements ===\n")
    f.write(f"Original Max Drawdown: {investment_results['max_drawdown_percent']:.2f}%\n")
    f.write(f"Drawdown Reduction: {investment_results['max_drawdown_percent'] - risk_managed_results['max_drawdown_percent']:.2f}%\n")
    f.write(f"Performance Ratio: {performance_ratio:.2f}x\n\n")
    
    f.write("=== Risk Management Techniques Applied ===\n")
    f.write("1. Position Sizing: Adjusted position sizes based on prediction confidence\n")
    f.write("2. Stop-Loss Protection: Implemented 5% fixed and 7% trailing stop losses\n")
    f.write("3. Volatility Adjustment: Reduced exposure during high volatility periods\n")
    f.write("4. Trend Following: Aligned positions with overall market trend\n")
    f.write("5. Profit Taking: Implemented 10% profit-taking threshold\n\n")
    
    f.write("=== Detailed Trading Actions ===\n")
    for action in risk_managed_results['actions']:
        f.write(f"{action}\n")

print("\n=== Risk Management Analysis Complete ===")
print("Enhanced trading strategy results and comparisons saved to visualizations directory")
print(f"Drawdown reduced from {investment_results['max_drawdown_percent']:.2f}% to {risk_managed_results['max_drawdown_percent']:.2f}%")

# ===== VISUALIZATION 6: INVESTMENT RETURNS =====
plt.figure(figsize=(15, 10))

# Plot portfolio value over time
plt.subplot(2, 1, 1)
plt.plot(investment_results['portfolio_values'], 'g-', linewidth=2)
plt.title('Portfolio Value Over Time (Starting with $100)', fontsize=16)
plt.xlabel('Trading Days', fontsize=12)
plt.ylabel('Portfolio Value ($)', fontsize=12)
plt.grid(True, alpha=0.3)

# Add buy/sell markers
buy_days = []
buy_values = []
sell_days = []
sell_values = []

for i, action in enumerate(investment_results['actions']):
    if 'BUY' in action:
        buy_days.append(i)
        buy_values.append(investment_results['portfolio_values'][i])
    elif 'SELL' in action:
        sell_days.append(i)
        sell_values.append(investment_results['portfolio_values'][i])

plt.scatter(buy_days, buy_values, color='blue', marker='^', s=100, label='Buy')
plt.scatter(sell_days, sell_values, color='red', marker='v', s=100, label='Sell')
plt.legend(fontsize=12)

# Plot price and predictions with buy/sell signals
plt.subplot(2, 1, 2)
plt.plot(y_test_orig[:, 0], 'k-', label='Actual Price', linewidth=2)
plt.plot(final_filtered_preds[:, 0], 'b--', label='Predicted Price', linewidth=2, alpha=0.7)

# Add buy/sell markers
plt.scatter(buy_days, [y_test_orig[day, 0] for day in buy_days], color='blue', marker='^', s=100, label='Buy Signal')
plt.scatter(sell_days, [y_test_orig[day, 0] for day in sell_days], color='red', marker='v', s=100, label='Sell Signal')

plt.title('Stock Price with Trading Signals', fontsize=16)
plt.xlabel('Trading Days', fontsize=12)
plt.ylabel('Stock Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('visualizations/investment_returns.png')
plt.close()

# Save detailed trading log
with open('visualizations/trading_log.txt', 'w') as f:
    f.write("=== Investment Trading Log ===\n")
    f.write(f"Initial Investment: ${investment_results['initial_investment']:.2f}\n")
    f.write(f"Final Portfolio Value: ${investment_results['final_value']:.2f}\n")
    f.write(f"Total Return: {investment_results['total_return_percent']:.2f}%\n")
    f.write(f"Annualized Return: {investment_results['annualized_return_percent']:.2f}%\n")
    f.write(f"Maximum Drawdown: {investment_results['max_drawdown_percent']:.2f}%\n\n")
    
    f.write("=== Detailed Trading Actions ===\n")
    for action in investment_results['actions']:
        f.write(f"{action}\n")

# ================== 11. Advanced Model Comparison ==================
# Here we implement a baseline model to compare against our advanced model
# This helps validate that our sophisticated approach is actually better

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Prepare data for baseline models (using the same window size)
X_train_2d = X_train.reshape(X_train.shape[0], -1)  # Flatten the window dimensions
X_test_2d = X_test.reshape(X_test.shape[0], -1)  # Flatten the window dimensions

# Train a linear regression baseline
lr_model = LinearRegression()
lr_model.fit(X_train_2d, y_train[:, 0])  # Predict first day only for simplicity
lr_preds = lr_model.predict(X_test_2d)

# Train a random forest baseline
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_2d, y_train[:, 0])  # Predict first day only for simplicity
rf_preds = rf_model.predict(X_test_2d)

# Convert predictions back to original scale
lr_preds_orig = target_scaler.inverse_transform(lr_preds.reshape(-1, 1)).flatten()
rf_preds_orig = target_scaler.inverse_transform(rf_preds.reshape(-1, 1)).flatten()

# Calculate metrics for baselines
lr_rmse = np.sqrt(mean_squared_error(y_test_orig[:, 0], lr_preds_orig))
rf_rmse = np.sqrt(mean_squared_error(y_test_orig[:, 0], rf_preds_orig))
deep_model_rmse = metrics_by_day[0]['RMSE']

print("\n=== Model Comparison ===")
print(f"Deep TCN Model RMSE: {deep_model_rmse:.2f}")
print(f"Random Forest RMSE: {rf_rmse:.2f}")
print(f"Linear Regression RMSE: {lr_rmse:.2f}")
print(f"Improvement over Linear Regression: {(1 - deep_model_rmse/lr_rmse) * 100:.2f}%")
print(f"Improvement over Random Forest: {(1 - deep_model_rmse/rf_rmse) * 100:.2f}%")

# ===== VISUALIZATION 7: MODEL COMPARISON =====
plt.figure(figsize=(15, 8))

# Plot predictions from all models
plt.plot(y_test_orig[:, 0], 'k-', label='Actual Price', linewidth=2.5)
plt.plot(final_filtered_preds[:, 0], 'b-', label=f'Deep TCN (RMSE: {deep_model_rmse:.2f})', linewidth=2)
plt.plot(rf_preds_orig, 'g--', label=f'Random Forest (RMSE: {rf_rmse:.2f})', linewidth=1.5, alpha=0.7)
plt.plot(lr_preds_orig, 'r:', label=f'Linear Regression (RMSE: {lr_rmse:.2f})', linewidth=1.5, alpha=0.7)

plt.title('Model Comparison', fontsize=16)
plt.xlabel('Trading Days', fontsize=12)
plt.ylabel('Stock Price ($)', fontsize=12)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)

plt.savefig('visualizations/model_comparison.png')
plt.close()

# Display summary report
print("\n=== SUMMARY REPORT ===")
print(f"Our model predicts that starting with $100 would result in ${investment_results['final_value']:.2f}")
print(f"This represents a {investment_results['total_return_percent']:.2f}% return over the test period")
print(f"The model outperformed simple baselines by up to {max((1 - deep_model_rmse/lr_rmse) * 100, (1 - deep_model_rmse/rf_rmse) * 100):.2f}%")
print(f"Maximum drawdown during trading: {investment_results['max_drawdown_percent']:.2f}%")
print(f"Trading summary: {len(buy_days)} buys, {len(sell_days)} sells")
print("\nVisualization and detailed logs saved to the 'visualizations' directory")

# Create final portfolio value plot for immediate display
plt.figure(figsize=(14, 7))
plt.plot(investment_results['portfolio_values'], 'g-', linewidth=2.5)
plt.title(f'Portfolio Value Over Time (Starting with $100)', fontsize=16)
plt.xlabel('Trading Days', fontsize=12)
plt.ylabel('Portfolio Value ($)', fontsize=12)
plt.grid(True, alpha=0.3)

# Add key metrics as text
plt.text(
    0.02, 0.85, 
    f'Final Value: ${investment_results["final_value"]:.2f}\n'
    f'Total Return: {investment_results["total_return_percent"]:.2f}%\n'
    f'Annualized Return: {investment_results["annualized_return_percent"]:.2f}%\n'
    f'Max Drawdown: {investment_results["max_drawdown_percent"]:.2f}%', 
    transform=plt.gca().transAxes,
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.8)
)

plt.tight_layout()
plt.savefig('visualizations/final_portfolio_value.png')
plt.show()  # This will display the plot if running in an interactive environmentment